<a href="https://colab.research.google.com/github/sajed-s/FASTA_Analysis/blob/main/downloading_fasta_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libreries

In [ ]:
import dendropy
from dendropy.interop import genbank
import pandas as pd
from Bio import SeqIO

# Importing the text file of gene IDs

In [ ]:
with open("E:\python\FMD\GI_A.txt", "r") as my_file:
    data = my_file.read()
    GIlist = data.replace('\n', ' ').split(" ")

data_part1 = GIlist[0:200]
data_part2 = GIlist[200:]

# Extracting the data

In [ ]:
gene_id = []
date = []
organism = []
title = []
journal = []
pubmed_id = []

gb_dna = genbank.GenBankDna(data_part1)
for gb_rec in gb_dna:
    gene_id.append(gb_rec.gi)
    date.append(gb_rec.create_date)
    organism.append(gb_rec.organism)
    for ref in gb_rec.references:
        pubmed_id.append(ref.pubmed_id)
        journal.append(ref.journal)
        title.append(ref.title)

gb_dna = genbank.GenBankDna(data_part2)
for gb_rec in gb_dna:
    gene_id.append(gb_rec.gi)
    date.append(gb_rec.create_date)
    organism.append(gb_rec.organism)
    for ref in gb_rec.references:
        pubmed_id.append(ref.pubmed_id)
        journal.append(ref.journal)
        title.append(ref.title)

# Saving the data to CSV files

In [ ]:
data_id_org_date = pd.DataFrame({'gene_id': gene_id, 'organism': organism, 'date': date})
data_pubmed_journal_title = pd.DataFrame({'journal': journal, 'pubmed_id': pubmed_id, 'title': title})
data_id_org_date.to_csv('data_id_org_date_a.csv')
data_pubmed_journal_title.to_csv('data_pubmed_journal_a.csv')

 # Downloading the strains

In [ ]:
def get_fasta_a(ids):
    for i in ids:
        yield 'strain_a', genbank.GenBankDna(ids=i)

sampled1 = open('sample_a1.fasta', 'w')
for species, recs in get_fasta_a([data_part1]):
    tn = dendropy.TaxonNamespace()
    char_mat = recs.generate_char_matrix(taxon_namespace=tn, gb_to_taxon_fn=lambda gb: tn.require_taxon(label='%s_%s' % (species, gb.accession)))
    char_mat.write_to_stream(sampled1, 'fasta')
sampled1.close()

sampled2 = open('sample_a2.fasta', 'w')
for species, recs in get_fasta_a([data_part2]):
    tn = dendropy.TaxonNamespace()
    char_mat = recs.generate_char_matrix(taxon_namespace=tn, gb_to_taxon_fn=lambda gb: tn.require_taxon(label='%s_%s' % (species, gb.accession)))
    char_mat.write_to_stream(sampled2, 'fasta')
sampled2.close()


# Merging the fasta files

In [ ]:
input_files = ["sample_a1.fasta", "sample_a2.fasta"]
output_file = "total_a.fasta"

with open(output_file, "w") as outfile:
    for input_file in input_files:
        for record in SeqIO.parse(input_file, "fasta"):
            SeqIO.write(record, outfile, "fasta")

print("Merged fasta files written to", output_file)
